In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as scisig
import scipy.optimize as opt
import scipy.io as sio
import simulation_utils as su
import analysis_utils as au
import importlib
import h5py

In [2]:
# Define some parameters

w0 = 55000*2*np.pi # Natural frequency of oscillator
P = 1*10**(-7) # Pressure in mbar
T = 293 # Environmental temperature
R = (0.083)*10**(-6) # Particle Radius
p = 2200 # Particle Denisty
dtn = 1*10**(-8) # Timestep
M = 4/3*np.pi*R**3*p # Particle mass
kb = 1.38*10**(-23) # Boltzmann constant
m = 29*1.67*10**(-27) # average mass of air particles
v = np.sqrt(8*1.38*10**(-23)*T/(np.pi*m)) # velocity of air particles
N0 = 2.43*10**(25) # Number density of air and ambient pressure
P0 = 1000 # Ambient pressure
N=N0*P/P0 # Number density of air at pressure P

y = (4/3*np.pi)*(m*v*N*R**2)/(M) # Damping from air
#y = 4.10568*1e-6*P*1e2/R # Antonio's damping from air

print(y)

Snn = 10**(-28) # Displacement noise

0.00029717080823618027


# Generating impulses and noise for a stationary transfer function oscillator

In [ ]:
# Simulation
# This block generates the data and saves it
# Calculating num_sims numbers of simulations with and without response
# Also generating average response to impulse and noise template

num_sims = 1000

# feedback damping values to use
yfb_l = np.array([10000, 1000, 100, 10, 1])*2*np.pi

for yfb in yfb_l:
    filename = 'D:/Simulation Data/240501/yfb' + str(round(yfb/2/np.pi))

    # Some set-up
    maxw = 5*10**5 # 500 kHz is enough to capture all dynamics
    numbins = int(2*5*10**4*100*2*np.pi/yfb) # This is to ensure the particle amplitude decays to thermal levels
    w = np.linspace(0, maxw*2*np.pi, numbins)
    
    # Generate impulse response
    time = np.linspace(0, numbins/5/10**5/2, numbins)
    ir = su.impulse_resp(time, time[int(numbins/2)], 10*10**(-10), y+yfb, w0) # Same impulse response every time so generate it here

    # Generate some random number seeds
    np.random.seed(85655219)
    random_nums = np.random.uniform(0, 1000000, 6*num_sims) # These are the seeds we will use
    
    # Set-up arrays for calculating average response (s in optimal filter/data to fit s to)
    avr_resp = np.zeros(numbins)
    
    # Now generate num_sims different thermally driven particles and add the impulse response
    for i in range(num_sims):
        randomlist, randomlist2, randomlist3 = su.generate_random_numbers(random_nums[3*i:3*(i+1)], Snn, numbins, maxw)
        x = su.generate_displacement(w, w0, y, yfb, randomlist, randomlist2, randomlist3, ir)

        # Save the time domain response
        mdict = {'x': x}
        sio.savemat(filename+'/Kicks/' + str(i) + '.mat', mdict)

        # Add to average response
        avr_resp += x

    # Save average response to use as template/template fit
    avr_resp /= num_sims
    mdict = {'x':avr_resp}
    sio.savemat(filename+'/Kicks/avr_resp.mat', mdict)

    # Generate nums_sims new thermally driven particles with no impulse response to create noise template and to run optimal filter on for comparison

    # Set-up array for noise template (J in optimal filter)
    PSD_noise = np.zeros(int(numbins/2)+1)

    # Create empty array for impulse response because we don't want and impulse
    ir = np.zeros(numbins)

    for i in range(num_sims, 2*num_sims):
        n = i-num_sims
        randomlist, randomlist2, randomlist3 = su.generate_random_numbers(random_nums[3*i:3*(i+1)], Snn, numbins, maxw)
        x = su.generate_displacement(w, w0, y, yfb, randomlist, randomlist2, randomlist3, ir)

        # Save the time domain response
        mdict = {'x': x}
        sio.savemat(filename+'/Noise/' + str(n) + '.mat', mdict)

        # Calculate PSD and add to J
        PSD = scisig.welch(x, fs = 10**6, nperseg = numbins)
        PSD_noise += PSD[1]

    # Save J
    PSD_noise /= num_sims
    PSD_freq = PSD[0]
    mdict = {'J':PSD_noise, 'frequency':PSD_freq}
    sio.savemat(filename+'/Noise/PSD.mat', mdict)


# Generating data for a non-stationary transfer function oscillator

In this first attempt I will try generating a stationary transfer function, reverse FTing it to time domain, create a modulated timebase then interpolate the displacement trace and finally FT back into the frequency domain. This should be the equivalent of modulating the central frequency of the oscillator. It is probably faster than doing everything in the time domain. First, I will try it on like 100 Hz data and try broadening with a sin modulation to 1 kHz or so. Then I will try it on lower damping data and time it to compare to generating stuff in the time domain.

In [19]:
importlib.reload(su)

num_sims = 1000

# I think saving J and average response the normal way is kind of pointless here but let's do it anyway
# feedback damping values to use
yfb_l = np.array([1000])*2*np.pi
iter_l = [0.0, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2]
for yfb in yfb_l:
    for iter in iter_l:
        filename = 'D:/Simulation Data/240529/Frequency modulation/yfb' + str(round(yfb/2/np.pi)) + '/' + str(iter)

        # Some set-up
        maxw = 5*10**5 # 500 kHz is enough to capture all dynamics
        numbins = int(2*5*10**4*100*2*np.pi/yfb) # This is to ensure the particle amplitude decays to thermal levels
        w = np.linspace(0, maxw*2*np.pi, numbins)

        # Generate some random number seeds
        np.random.seed(85655219)
        random_nums = np.random.uniform(0, 1000000, 6*num_sims) # These are the seeds we will use
        np.random.seed(85634442)
        phase = np.random.uniform(0, 1, 2*num_sims)
        

        # Set-up arrays for calculating average response (s in optimal filter/data to fit s to)
        avr_resp = np.zeros(numbins)
        
        # Now generate num_sims different thermally driven particles and add the impulse response
        time = np.linspace(0, numbins/5/10**5/2, numbins)
        for i in range(0, num_sims):
            n = i
            randomlist, randomlist2, randomlist3 = su.generate_random_numbers(random_nums[3*i:3*(i+1)], Snn, numbins, maxw)
            fm = su.generate_sawtooth_frequency_modulation_impulse(time, iter, phase[i])
            ir = su.impulse_resp_fm(time, time[int(numbins/2)], 10*10**(-10), y, yfb, w0, fm)
            x = su.generate_displacement_fm(w, w0, y, yfb, M, T, randomlist, randomlist2, randomlist3, ir, iter, phase[i])

            # Save time domain response
            mdict = {'x': x, 'ir': ir, 'fm': fm} # saving timetrace, true impulse and true frequency modulation
            su.save_data_hdf5(filename+'/Kicks/' + str(n) + '.hdf5', mdict)

            # Add to average response
            avr_resp += x

        # Save average response to use as template/template fit
        avr_resp /= num_sims
        mdict = {'x':avr_resp}
        su.save_data_hdf5(filename+'/Kicks/avr_resp.hdf5', mdict)

        # Generate 1000 new thermally driven particles with no impulse response to create noise template and to run optimal filter on for comparison

        # Set-up array for noise template (J in optimal filter)
        PSD_noise = np.zeros(int(numbins/2)+1)

        # Create empty array for impulse response because we don't want and impulse
        ir = np.zeros(numbins)

        for i in range(num_sims, 2*num_sims):
            n = i-num_sims
            randomlist, randomlist2, randomlist3 = su.generate_random_numbers(random_nums[3*i:3*(i+1)], Snn, numbins, maxw)
            x = su.generate_displacement_fm(w, w0, y, yfb, M, T, randomlist, randomlist2, randomlist3, ir, iter, phase[i])
            mdict = {'x': x, 'ir': ir, 'fm': fm} # saving timetrace, true impulse and true frequency modulation
            su.save_data_hdf5(filename+'/Noise/' + str(n) + '.hdf5', mdict)
            # Calculate PSD and add to J
            PSD = scisig.welch(x, fs = 10**6, nperseg = numbins, window = 'boxcar')
            PSD_noise += PSD[1]

        # Save J
        PSD_noise /= num_sims
        PSD_freq = PSD[0]
        mdict = {'J':PSD_noise, 'frequency':PSD_freq}
        su.save_data_hdf5(filename+'/Noise/PSD.hdf5', mdict)